In [ ]:
from dotenv import load_dotenv
import os
from env import * 


load_dotenv()


# ALPACA_KEY = os.getenv("APCA_API_KEY_ID")
# ALPACA_SECRET_KEY = os.getenv("APCA_API_SECRET_KEY")
# print(ALPACA_KEY, ALPACA_SECRET_KEY)

In [2]:
from alpaca.data import StockHistoricalDataClient

client = StockHistoricalDataClient(ALPACA_KEY, ALPACA_SECRET_KEY)
# client = StockHistoricalDataClient()

In [13]:
print(client)

In [14]:
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import polars as pl

In [25]:
SYMBOL = "SPY"
# eastern = ZoneInfo("")
START_DATE = datetime(2020, 1, 1, 0, 0, tzinfo=timezone.utc)
END_DATE = datetime(2025, 12, 30, 0, 0, tzinfo=timezone.utc) 
chunk_size = timedelta(days=30)
# request = StockBarsRequest(symbol_or_symbols=symbol, start=startdate, end=enddate, timeframe=TimeFrame.Minute)
# print(request)

In [11]:
# startdate = str(startdate).replace(" ", "")
# enddate = str(enddate).replace(" ", "")
# print(startdate, enddate)
# print(startdate.date())




In [23]:
DATA_ROOT = "./data"
def save_partition(df: pl.DataFrame, symbol: str):
    if df.is_empty():
        return

    df = df.with_columns([
        pl.col("timestamp").dt.year().alias("year"),
        pl.col("timestamp").dt.month().alias("month"),
    ])

    for (year, month), part in df.partition_by(["year", "month"], as_dict=True).items():

        # DROP helper columns BEFORE saving / merging
        part = part.drop(["year", "month"])

        path = f"{DATA_ROOT}/symbol={symbol}/year={year}/month={month:02d}.parquet"
        os.makedirs(os.path.dirname(path), exist_ok=True)

        if os.path.exists(path):
            old = pl.read_parquet(path)

            # ensure same column order
            part = part.select(old.columns)

            part = pl.concat([old, part]).unique(subset=["timestamp"])

        part.write_parquet(path, compression="zstd")
        print(f"Saved {len(part)} rows → {path}")



In [26]:
chunk_start = START_DATE
chunk_size = timedelta(days=30)

while chunk_start < END_DATE:
    chunk_end = min(chunk_start + chunk_size, END_DATE)

    print(f"\nDownloading {chunk_start.date()} → {chunk_end.date()}")

    request = StockBarsRequest(
        symbol_or_symbols=[SYMBOL],
        start=chunk_start,
        end=chunk_end,
        timeframe=TimeFrame.Minute,
    )

    bars = client.get_stock_bars(request)
    df = bars.df

    if df.empty:
        print("No data returned")
        chunk_start = chunk_end
        continue

    df = df.reset_index()   # makes symbol + timestamp columns
    df = pl.from_pandas(df)

    save_partition(df, SYMBOL)

    chunk_start = chunk_end

print("\n✅ Done")


Saved 15628 rows → ./data/symbol=SPY/year=2020/month=01.parquet

Saved 16437 rows → ./data/symbol=SPY/year=2020/month=01.parquet
Saved 15591 rows → ./data/symbol=SPY/year=2020/month=02.parquet

Saved 19462 rows → ./data/symbol=SPY/year=2020/month=03.parquet

Saved 20408 rows → ./data/symbol=SPY/year=2020/month=03.parquet
Saved 18233 rows → ./data/symbol=SPY/year=2020/month=04.parquet

Saved 19153 rows → ./data/symbol=SPY/year=2020/month=04.parquet
Saved 17635 rows → ./data/symbol=SPY/year=2020/month=05.parquet

Saved 17859 rows → ./data/symbol=SPY/year=2020/month=06.parquet

Saved 19662 rows → ./data/symbol=SPY/year=2020/month=06.parquet
Saved 15913 rows → ./data/symbol=SPY/year=2020/month=07.parquet

Saved 18328 rows → ./data/symbol=SPY/year=2020/month=07.parquet
Saved 15046 rows → ./data/symbol=SPY/year=2020/month=08.parquet

Saved 16586 rows → ./data/symbol=SPY/year=2020/month=08.parquet
Saved 15182 rows → ./data/symbol=SPY/year=2020/month=09.parquet

Saved 17788 rows → ./data/symb

In [51]:
len(data)

314

In [52]:
data

open      high      low   close  \
symbol timestamp                                                      
SPY    2023-01-03 09:00:00+00:00  384.00  386.8300  384.000  386.83   
       2023-01-03 10:00:00+00:00  386.63  386.6400  385.620  386.06   
       2023-01-03 11:00:00+00:00  385.90  386.0000  383.520  383.96   
       2023-01-03 12:00:00+00:00  383.95  384.4400  383.340  384.31   
       2023-01-03 13:00:00+00:00  383.53  386.7204  382.770  384.69   
...                                  ...       ...      ...     ...   
       2023-01-31 20:00:00+00:00  404.50  406.5300  403.645  406.45   
       2023-01-31 21:00:00+00:00  406.45  406.4800  405.690  405.95   
       2023-01-31 22:00:00+00:00  405.92  406.0800  405.790  406.02   
       2023-01-31 23:00:00+00:00  406.02  406.0200  405.330  405.55   
       2023-02-01 00:00:00+00:00  405.51  405.9500  405.010  405.09   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
SPY    2023-01-03 09:00:00+00:00    146402.0       1069.0  386.296811  
       2023-01-03 10:00:00+00:00    115731.0        581.0  385.945177  
       2023-01-03 11:00:00+00:00    129478.0       1010.0  384.514164  
       2023-01-03 12:00:00+00:00    401021.0       2385.0  383.943566  
       2023-01-03 13:00:00+00:00    897999.0       5822.0  384.216894  
...                                      ...          ...         ...  
       2023-01-31 20:00:00+00:00  28458661.0     175325.0  405.239334  
       2023-01-31 21:00:00+00:00  19788546.0      11153.0  406.328586  
       2023-01-31 22:00:00+00:00    128748.0       1274.0  405.986075  
       2023-01-31 23:00:00+00:00    123960.0       1262.0  405.666846  
       2023-02-01 00:00:00+00:00     99592.0       1293.0  405.230204  

[314 rows x 7 columns]

In [ ]:
from pathlib import Path

output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# filename = f"./{output_dir}/stock_trades_{symbol}.parquet"
# print(filename)
# data.to_parquet(filename)
# print("Saved to parquet ", filename)

./output/stock_trades_SPY.parquet
Saved to parquet  ./output/stock_trades_SPY.parquet


# Get smallest tick

In [ ]:
from datetime import datetime
from alpaca.data.requests import StockTradesRequest
from alpaca.data.historical import StockHistoricalDataClient

symbol = "AAPL"
start = datetime(2025, 1, 5, 9, 30)  # market open
end   = datetime(2025, 1, 5, 16, 0)  # market close

request = StockTradesRequest(
    symbol_or_symbols=symbol,
    start=start,
    end=end,
    
    # limit=None means fetch *all* within range
)

response = client.get_stock_trades(request)
df_trades = response.df    # pandas DataFrame

print(df_trades.head())


Empty DataFrame
Columns: []
Index: []


# Crypto market

In [30]:
from alpaca.data.historical import CryptoHistoricalDataClient

# No keys required for crypto data
client = CryptoHistoricalDataClient()

In [62]:
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD"],
  timeframe=TimeFrame.Day,
  start=datetime(2022, 1, 1),
  end=datetime(2025, 1, 1)
)

In [63]:
# Retrieve daily bars for Bitcoin in a DataFrame and printing it
btc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
btc_bars.df

open        high           low  \
symbol  timestamp                                                         
BTC/USD 2022-01-01 00:00:00+00:00  46212.0600  47950.0000  46208.240000   
        2022-01-02 00:00:00+00:00  47725.9800  47992.0000  46668.460000   
        2022-01-03 00:00:00+00:00  47297.6100  47568.9500  45700.830000   
        2022-01-04 00:00:00+00:00  46461.2800  47527.7800  45547.550000   
        2022-01-05 00:00:00+00:00  45823.9900  47066.7000  42460.000000   
...                                       ...         ...           ...   
        2024-12-28 00:00:00+00:00  94226.3000  95496.2175  94015.050000   
        2024-12-29 00:00:00+00:00  95106.3250  95149.0000  92871.750000   
        2024-12-30 00:00:00+00:00  93586.2650  94827.4550  91348.456517   
        2024-12-31 00:00:00+00:00  92628.9215  96119.0200  91934.610000   
        2025-01-01 00:00:00+00:00  93381.5825  94976.9360  92772.200000   

                                       close       volume  trade_count  \
symbol  timestamp                                                        
BTC/USD 2022-01-01 00:00:00+00:00  47727.800   488.529238      25254.0   
        2022-01-02 00:00:00+00:00  47302.830   459.639797      21844.0   
        2022-01-03 00:00:00+00:00  46422.290   722.373139      28779.0   
        2022-01-04 00:00:00+00:00  45830.460   890.039831      30410.0   
        2022-01-05 00:00:00+00:00  43437.260  1632.657507      50866.0   
...                                      ...          ...          ...   
        2024-12-28 00:00:00+00:00  95164.460     1.203702        116.0   
        2024-12-29 00:00:00+00:00  93607.075     0.742278        164.0   
        2024-12-30 00:00:00+00:00  92666.990     5.967735        336.0   
        2024-12-31 00:00:00+00:00  93470.520     4.599376        199.0   
        2025-01-01 00:00:00+00:00  94455.287     0.484563        106.0   

                                           vwap  
symbol  timestamp                                
BTC/USD 2022-01-01 00:00:00+00:00  47158.127380  
        2022-01-02 00:00:00+00:00  47265.366239  
        2022-01-03 00:00:00+00:00  46660.767085  
        2022-01-04 00:00:00+00:00  46436.794832  
        2022-01-05 00:00:00+00:00  45061.865871  
...                                         ...  
        2024-12-28 00:00:00+00:00  94584.605243  
        2024-12-29 00:00:00+00:00  94275.522390  
        2024-12-30 00:00:00+00:00  92614.696280  
        2024-12-31 00:00:00+00:00  94491.109377  
        2025-01-01 00:00:00+00:00  94126.504821  

[1097 rows x 7 columns]

# Analyze parquet


In [30]:
import pandas as pd

In [36]:
df = pd.read_parquet(filename)
# df = pd.read_parquet(filename, columns=["timestamp", "volume"])

In [37]:
df

open    high       low   close  \
symbol timestamp                                                     
SPY    2023-01-03 05:00:00+00:00  384.37  386.43  377.8310  380.82   
       2023-01-04 05:00:00+00:00  383.18  385.88  380.0000  383.76   
       2023-01-05 05:00:00+00:00  381.72  381.84  378.7600  379.38   
       2023-01-06 05:00:00+00:00  382.61  389.25  379.4127  388.08   
       2023-01-09 05:00:00+00:00  390.37  393.70  387.6700  387.86   
...                                  ...     ...       ...     ...   
       2023-12-22 05:00:00+00:00  473.86  475.38  471.7000  473.65   
       2023-12-26 05:00:00+00:00  474.07  476.58  473.9900  475.65   
       2023-12-27 05:00:00+00:00  475.44  476.66  474.8900  476.51   
       2023-12-28 05:00:00+00:00  476.88  477.55  476.2600  476.69   
       2023-12-29 05:00:00+00:00  476.49  477.03  473.3000  475.31   

                                       volume  trade_count        vwap  
symbol timestamp                                                        
SPY    2023-01-03 05:00:00+00:00   74850731.0     590240.0  380.745269  
       2023-01-04 05:00:00+00:00   85934098.0     632808.0  383.265078  
       2023-01-05 05:00:00+00:00   76970456.0     530899.0  380.035868  
       2023-01-06 05:00:00+00:00  104026233.0     685781.0  385.902328  
       2023-01-09 05:00:00+00:00   73978071.0     549428.0  390.656766  
...                                       ...          ...         ...  
       2023-12-22 05:00:00+00:00   67160419.0     486178.0  473.951699  
       2023-12-26 05:00:00+00:00   55386952.0     348986.0  475.354905  
       2023-12-27 05:00:00+00:00   68000811.0     425538.0  475.741039  
       2023-12-28 05:00:00+00:00   77158117.0     374241.0  476.950071  
       2023-12-29 05:00:00+00:00  122288349.0     604838.0  475.210777  

[250 rows x 7 columns]

In [38]:
df.head()

open    high       low   close  \
symbol timestamp                                                     
SPY    2023-01-03 05:00:00+00:00  384.37  386.43  377.8310  380.82   
       2023-01-04 05:00:00+00:00  383.18  385.88  380.0000  383.76   
       2023-01-05 05:00:00+00:00  381.72  381.84  378.7600  379.38   
       2023-01-06 05:00:00+00:00  382.61  389.25  379.4127  388.08   
       2023-01-09 05:00:00+00:00  390.37  393.70  387.6700  387.86   

                                       volume  trade_count        vwap  
symbol timestamp                                                        
SPY    2023-01-03 05:00:00+00:00   74850731.0     590240.0  380.745269  
       2023-01-04 05:00:00+00:00   85934098.0     632808.0  383.265078  
       2023-01-05 05:00:00+00:00   76970456.0     530899.0  380.035868  
       2023-01-06 05:00:00+00:00  104026233.0     685781.0  385.902328  
       2023-01-09 05:00:00+00:00   73978071.0     549428.0  390.656766

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 250 entries, ('SPY', Timestamp('2023-01-03 05:00:00+0000', tz='UTC')) to ('SPY', Timestamp('2023-12-29 05:00:00+0000', tz='UTC'))
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         250 non-null    float64
 1   high         250 non-null    float64
 2   low          250 non-null    float64
 3   close        250 non-null    float64
 4   volume       250 non-null    float64
 5   trade_count  250 non-null    float64
 6   vwap         250 non-null    float64
dtypes: float64(7)
memory usage: 24.7+ KB


In [27]:
import polars as pl

df = pl.read_parquet("./data/symbol=SPY/**/*.parquet")
print(df.shape)
print(df.head())


(1238488, 9)
shape: (5, 9)
┌────────┬─────────────────────┬────────┬────────┬───┬────────┬─────────┬─────────────┬────────────┐
│ symbol ┆ timestamp           ┆ open   ┆ high   ┆ … ┆ close  ┆ volume  ┆ trade_count ┆ vwap       │
│ ---    ┆ ---                 ┆ ---    ┆ ---    ┆   ┆ ---    ┆ ---     ┆ ---         ┆ ---        │
│ str    ┆ datetime[ns, UTC]   ┆ f64    ┆ f64    ┆   ┆ f64    ┆ f64     ┆ f64         ┆ f64        │
╞════════╪═════════════════════╪════════╪════════╪═══╪════════╪═════════╪═════════════╪════════════╡
│ SPY    ┆ 2020-01-02 17:15:00 ┆ 323.07 ┆ 323.1  ┆ … ┆ 323.08 ┆ 91603.0 ┆ 917.0       ┆ 323.044572 │
│        ┆ UTC                 ┆        ┆        ┆   ┆        ┆         ┆             ┆            │
│ SPY    ┆ 2020-01-16 00:38:00 ┆ 328.76 ┆ 328.76 ┆ … ┆ 328.75 ┆ 1125.0  ┆ 4.0         ┆ 328.759091 │
│        ┆ UTC                 ┆        ┆        ┆   ┆        ┆         ┆             ┆            │
│ SPY    ┆ 2020-01-06 12:56:00 ┆ 321.26 ┆ 321.26 ┆ … ┆ 321.26 ┆ 

# Aggregation

In [ ]:
import polars as pl
from datetime import datetime